In [12]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import pandas as pd

# Load the pre-trained model
model_name = "SanketJadhav/PlantDiseaseClassifier-Resnet50"
extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

# Assuming you have an image file called "image.jpg" that you want to classify
image_path = r"C:\Users\ayub_\OneDrive\Documents\test1_img.jpg"

# Assuming num_labels is the number of output classes in your model
num_labels = model.config.num_labels

# Create a mapping from class index to label
class_to_label = {i: f"Disease {i+1}" for i in range(num_labels)}



from PIL import Image
import numpy as np

# Load the image using PIL
image = Image.open(image_path)

# Convert the image to a numpy array
image_array = np.array(image)

# Use the extractor to preprocess the image array
inputs = extractor(images=image_array, return_tensors="pt")

# Pass the preprocessed inputs through the model
outputs = model(**inputs)

# Get the predicted class and probability
predicted_class = outputs.logits.argmax().item()
predicted_probability = outputs.logits.softmax(dim=1)[0, predicted_class].item()

# Create a mapping from class index to label
class_to_label = {i: model.config.id2label[i] for i in range(model.config.num_labels)}

# Get the disease label based on the predicted class
predicted_disease = class_to_label[predicted_class]

# Load the disease solution dataframe
disease_solution = pd.read_csv(r"C:\Users\ayub_\disease_solution.csv")

# Find the corresponding solution for the predicted disease
solution_row = disease_solution[disease_solution["Disease"] == predicted_disease]

# Get the solution from the dataframe
predicted_solution = solution_row["Solution"].values[0]

# Print the results
print(f"Predicted class: {predicted_class}")
print(f"Predicted disease: {predicted_disease}")
print(f"Predicted probability: {predicted_probability}")
print(f"Predicted solution: {predicted_solution}")

C:\Users\ayub_\anaconda3\lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


Predicted class: 0
Predicted disease: Apple___Apple_scab
Predicted probability: 0.4529123902320862
Predicted solution: Apply fungicides and prune infected branches.


In [2]:
from flask import Flask, request, jsonify
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import numpy as np

# Load the pre-trained model
model_name = "SanketJadhav/PlantDiseaseClassifier-Resnet50"
extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

# Create a mapping from class index to label
class_to_label = {i: f"Disease {i+1}" for i in range(model.config.num_labels)}

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Get the image data from the request
    image_file = request.files['image']
    image = Image.open(image_file)
    image_array = np.array(image)

    # Preprocess the image
    inputs = extractor(images=image_array, return_tensors="pt")
    outputs = model(**inputs)

    # Get the predicted class and probability
    predicted_class = outputs.logits.argmax().item()
    predicted_probability = outputs.logits.softmax(dim=1)[0, predicted_class].item()

    # Get the disease label based on the predicted class
    predicted_disease = class_to_label[predicted_class]

    # Return the prediction results as JSON response
    response = {
        'predicted_class': predicted_class,
        'predicted_disease': predicted_disease,
        'predicted_probability': predicted_probability
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
